In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True, # chain의 prompt 로그를 확인할 수 있음
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [15]:
chain.predict(question="I live is Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live is Seoul

> Finished chain.


"That's great! Seoul is the capital and largest city of South Korea. It is known for its vibrant culture, modern architecture, and delicious food. There are many attractions to explore in Seoul, such as Gyeongbokgung Palace, N Seoul Tower, Myeongdong shopping district, and the Han River. It's a city that seamlessly blends tradition and modernity. What do you enjoy most about living in Seoul?"

In [16]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

In [18]:
memory.load_memory_variables({})

{'history': "System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them. Nico mentions that they live in Seoul, and the AI responds by providing information about Seoul's culture, architecture, and attractions. The AI then asks Nico what they enjoy most about living in Seoul.\nHuman: What is my name?\nAI: I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."}

In [17]:
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})